# Value Extraction from Financial Statement Documents

Here I have used Regular Expressions to extract the data from the text files 

In [103]:
#importing libraries
import os
import json
import re
import xlsxwriter
import pandas as pd

#taking the file name in the order its been placed in expected format of submission file(i.e. Result.csv)
df=pd.read_csv(r'C:\Users\avinashexpc\Desktop\NLP\final\HCL ML Challenge\Results.csv')

#Writing on excel file which will be submission file in future  
outWorkbook=xlsxwriter.Workbook("C:\\Users\\avinashexpc\\Desktop\\NLP\\final\\HCL ML Challenge\\mu_res.xlsx")
outsheet=outWorkbook.add_worksheet()
outsheet.write(0,0,"Filename")
outsheet.write(0,1,"Extracted Values")
p=0
multi='C:\\Users\\avinashexpc\\Desktop\\NLP\\final\\HCL ML Challenge\\HCL ML Challenge Dataset\\'
for fil in df["Filename"]:
    
    #finding from where to start data extraction from each file
    with open(multi+fil+'.txt') as f:
        lines=f.readlines()
        pattern=re.compile(r'(Notes\s+)|(2020\s\s+)|(2019\s\s+)|(2018\s\s+)+.*')
        k=0
        for line in lines:
            line=line.strip()
            matches=pattern.findall(line)
            if matches!=[]:
                break
            #if(line==)
            k+=1
            
        #if pattern match is not found then assume data is not appropriate for this pattern 
        #go to next file
        if matches==[]:
            p+=1
            print(fil)
            print("^^^^^^^^^^0^^^^^^^^")
            outsheet.write(p,0,fil)
            continue
            
            
        #print(line)
        a=line.find('2018')
        b=line.find('2019')
        c=line.find('2020')
        print(fil)
        #print(b)
        
        
        #case 1: when balance sheet only have 2019 column
        if a==-1 and c==-1:
            
            
            #creating a pattern for each row of the columns
            pattern=re.compile(r'(([0-9a-zA-ZÂ£():|]+\s)+)\s+([0-9(),.-]+)\s+')
            lst=[]
            ans={}
            with open(multi+fil+'.txt') as f:
                               lines=f.readlines()
                               i=0
                               for line in lines:
                                    if line=='\n':
                                        continue 
                                    matches=pattern.findall(line)
                                    if matches!=[]:
                                        if matches[0][0]!='Notes ':
                                            it0=matches[0][0]
                                            it1=matches[0][2]
                                            it1=it1.replace("-","nan")
                                            it1=it1.replace("(","-")
                                            it1=it1.replace(")","")
                                            #it0=it0.replace("Â","")
                                            it1=it1.replace(",","")
                                            matches=[(it0,matches[0][1],it1)]
                                            #print(matches)
                                            ans[i]=matches
                                            lst.append(i)
                                    i=i+1
                                    
            #if pattern match is not found then assume data is not appropriate for this pattern 
            #go to next file                        
            if ans=={}:
               # print("WRONG")
                p+=1
                print(fil)
                print("^^^^^^^^^^0^^^^^^^^")
                outsheet.write(p,0,fil)
                continue
            
            #finding the range of lines or rows need to be extracted (i.e. range(a,b+1))
            a=min(k+2,min(lst))
            b=max(lst)
            
            #storing extracted data into dictionary(i.e. res)
            res={}
            abc=ans.keys()
            for i in range(a,b+1):
                if lines[i]=='\n':
                    continue 
                if i in ans:
                    res[ans[i][0][0]]=ans[i][0][2]
                else:
                    abc=re.findall(r'(([0-9a-zA-ZÂ£:()-|]+\s)+)',lines[i])
                    res[abc[0][0]]='nan'
            
            #converting dictionary(i.e res) into string (i.e. ltr)
            ltr="{"
            for i in res:
                ltr+='"'+str(i)[:-1]+'": '+'"'+str(res[i])+'", '
            ltr=ltr[:-2]
            ltr+="}"
            p+=1
            outsheet.write(p,0,fil)
            print(json.loads(ltr))
            outsheet.write(p,1,json.dumps(json.loads(ltr)))
            print("^^^^^^^^^^1^^^^^^^^")
            continue
            
        #case 2: when balance sheet not have 2019 column
        if b==-1:
            
            #creating a pattern for each row of the columns
            pattern=re.compile(r'(([0-9a-zA-ZÂ£():|]+\s)+)\s+([0-9(),.-]+)\s+')
            lst=[]
            ans={}
            with open(multi+fil+'.txt') as f:
                               lines=f.readlines()
                               i=0
                               for line in lines:
                                    if line=='\n':
                                        continue 
                                    matches=pattern.findall(line)
                                    if matches!=[]:
                                        if matches[0][0]!='Notes ':
                                            it0=matches[0][0]
                                            #it0=it0.replace("Â","")
                                            matches=[(it0,matches[0][1],"nan")]
                                            #print(matches)
                                            ans[i]=matches
                                            lst.append(i)
                                    i=i+1
                                    
            #if pattern match is not found then assume data is not appropriate for this pattern 
            #go to next file                        
            if ans=={}:
               # print("WRONG")
                p+=1
                print(fil)
                print("^^^^^^^^^^0^^^^^^^^")
                outsheet.write(p,0,fil)
                continue
            
            #finding the range of lines or rows need to be extracted (i.e. range(a,b+1))
            a=min(k+2,min(lst))
            b=max(lst)
            
            #storing extracted data into dictionary(i.e. res)
            res={}
            abc=ans.keys()
            for i in range(a,b+1):
                if lines[i]=='\n':
                    continue 
                if i in ans:
                    res[ans[i][0][0]]=ans[i][0][2]
                else:
                    abc=re.findall(r'(([0-9a-zA-ZÂ£:()-|]+\s)+)',lines[i])
                    res[abc[0][0]]='nan'
                    
            
            #converting dictionary(i.e res) into string (i.e. ltr)
            ltr="{"
            for i in res:
                ltr+='"'+str(i)[:-1]+'": '+'"'+str(res[i])+'", '
            ltr=ltr[:-2]
            ltr+="}"
            p+=1
            outsheet.write(p,0,fil)
            print(json.loads(ltr))
            outsheet.write(p,1,json.dumps(json.loads(ltr)))
            print("^^^^^^^^^^2^^^^^^^^")
            continue
        
        #case 3: when balance sheet has two column among which one is 2019 column
        
        #finding which column is 2019 column
        #if d=1 means 1st column is 2019 else if d=2 2nd column is 2019 column 
        d=0
        if a==-1:
            if b>c:
                d=2
            else:
                d=1
        if c==-1:
            if b>a:
                d=2
            else:
                d=1  
                
        #creating a pattern for each row of the columns
        pattern=re.compile(r'(([0-9a-zA-ZÂ£():|]+\s)+)\s+([0-9(),.-]+)\s+([0-9(),.-]+)')
        lst=[]
        ans={}
        with open(multi+fil+'.txt') as f:
                           lines=f.readlines()
                           i=0
                           for line in lines:
                                if line=='\n':
                                    continue 
                                matches=pattern.findall(line)
                                if matches!=[]:
                                    if matches[0][0]!='Notes ':
                                        it0=matches[0][0]
                                        it1=matches[0][2]
                                        it2=matches[0][3]
                                        it1=it1.replace("-","nan")
                                        it2=it2.replace("-","nan")
                                        it1=it1.replace("(","-")
                                        it2=it2.replace("(","-")
                                        it1=it1.replace(")","")
                                        it2=it2.replace(")","")
                                        #it0=it0.replace("Â","")
                                        it1=it1.replace(",","")
                                        it2=it2.replace(",","")
                                        matches=[(it0,matches[0][1],it1,it2)]
                                        #print(matches)
                                        ans[i]=matches
                                        lst.append(i)
                                i=i+1
        #if pattern match is not found then assume data is not appropriate for this pattern 
        #go to next file 
        if ans=={}:
           # print("WRONG")
            p+=1
            print(fil)
            print("^^^^^^^^^^0^^^^^^^^")
            outsheet.write(p,0,fil)
            continue
        
        
        #finding the range of lines or rows need to be extracted (i.e. range(a,b+1))
        a=min(k+2,min(lst))
        b=max(lst)
        
        
        #storing extracted data into dictionary(i.e. res)
        res={}
        abc=ans.keys()
        for i in range(a,b+1):
            if lines[i]=='\n':
                continue 
            if i in ans:
                res[ans[i][0][0]]=ans[i][0][d+1]
            else:
                abc=re.findall(r'(([0-9a-zA-ZÂ£:()-|]+\s)+)',lines[i])
                res[abc[0][0]]='nan'
        
        
        #converting dictionary(i.e res) into string (i.e. ltr)
        ltr="{"
        for i in res:
            ltr+='"'+str(i)[:-1]+'": '+'"'+str(res[i])+'", '
        ltr=ltr[:-2]
        ltr+="}"
        p+=1
        outsheet.write(p,0,fil)
        print(json.loads(ltr))
        outsheet.write(p,1,json.dumps(json.loads(ltr)))
        print("^^^^^^^^^3^^^^^^^^^")
outWorkbook.close()



X8XUG1JD
{'Current Assets': '312', 'Creditors: amounts falling due within one year': '-59', 'Net current assets (liabilities)': '253', 'Total assets less current liabilities': '253', 'Total net assets (liabilities)': '253', 'Capital and reserves': '253'}
^^^^^^^^^3^^^^^^^^^
X8XX0C57
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8XX02AX
{'Net assets': '0', 'Issued share capital': 'nan', '1 Ordinary Share 0 of Â£ each': '0', 'Total Shareholder funds': '0'}
^^^^^^^^^3^^^^^^^^^
X8XX0ANN
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8XX0BU1
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of Â£ each': '100', 'Total Shareholder funds': '

{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of Â£ each': '100', 'Total Shareholder funds': '100'}
^^^^^^^^^3^^^^^^^^^
X8XZICQX
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8XZIDG2
{'Called up share capital not paid': 'nan', 'Fixed Assets': 'nan', 'Current Assets': 'nan', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '0', 'Total net assets (liabilities)': '0', 'Capital and reserves': '0'}
^^^^^^^^^^1^^^^^^^^
X8XZIDMH
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8XZIE17
{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 

X8XZIQ16
{'Current Assets': '22560', 'Creditors: amounts falling due within one year': '-2038', 'Net current assets (liabilities)': '20522', 'Total assets less current liabilities': '20522', 'Total net assets (liabilities)': '20522', 'Capital and reserves': '20522'}
^^^^^^^^^3^^^^^^^^^
X8XZIQEG
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8XZIR1E
{'Fixed Assets': '46754', 'Current Assets': '8017', 'Creditors: amounts falling due within one year': '-65151', 'Net current assets (liabilities)': '-57134', 'Total assets less current liabilities': '-10380', 'Provisions for liabilities': '-7458', 'Accruals and deferred income': '-1200', 'Total net assets (liabilities)': '-19038', 'Capital and reserves': '-19038'}
^^^^^^^^^3^^^^^^^^^
X8XZIR3U
{'Fixed Assets': '8326', 'Current Assets': '17380', 'Creditors: amounts falling due within one year'

{'Called up share capital not paid': '1', 'Fixed Assets': '3334', 'Current Assets': 'nan', 'Prepayments and accrued income': '419', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '419', 'Total assets less current liabilities': '3754', 'Creditors: amounts falling due after more than one year': '-3754', 'Provisions for liabilities': '0', 'Accruals and deferred income': '0', 'Total net assets (liabilities)': '0', 'Capital and reserves': '0'}
^^^^^^^^^^1^^^^^^^^
X8Y4QNPL
{'Called up share capital not paid': 'nan', 'Fixed Assets': 'nan', 'Current Assets': 'nan', 'Prepayments and accrued income': 'nan', 'Creditors: amounts falling due within one year': 'nan', 'Net current assets (liabilities)': 'nan', 'Total assets less current liabilities': 'nan', 'Creditors: amounts falling due after more than one year': 'nan', 'Provisions for liabilities': 'nan', 'Accruals and deferred income': 'nan', 'Total net assets (liabilities)': 'nan', 'Capital and reserve

X8Y4R7MZ
{'Called up share capital not paid': '0', 'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Authorised share capital': 'nan', '0 Ordinary of Â£1 each': 'nan', 'Issued share capital': 'nan', '100 Ordinary of Â£1 each': '100', 'Total shareholder funds': '100'}
^^^^^^^^^3^^^^^^^^^
X8Y4R8QG
{'Called up share capital not paid': '98', 'Fixed Assets': '173395', 'Current Assets': '79129', 'Creditors: amounts falling due within one year': 'nan', 'Net current assets (liabilities)': '1884', 'Total assets less current liabilities': '175377', 'Creditors: amounts falling due after more than one': 'nan', '(56)': 'nan', 'year': 'nan', 'Total net assets (liabilities)': '175321', 'Capital and reserves': '175321'}
^^^^^^^^^3^^^^^^^^^
X8Y4R8UH
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8Y4R014
{'Fixed Assets': '8966', 'Current 

...

X8Y28MNT
{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', '100 Ordinary Shares 1 of Â£ each': '100', 'Total Shareholder funds': '100'}
^^^^^^^^^3^^^^^^^^^
X8Y28MOH
{'Fixed Assets': 'nan', 'Current Assets': 'nan', 'Prepayments and accrued income': 'nan', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '0', 'Creditors: amounts falling due after more than one year': '0', 'Provisions for liabilities': '0', 'Accruals and deferred income': '0', 'Total net assets (liabilities)': '0', 'Reserves': '0'}
^^^^^^^^^3^^^^^^^^^
X8Y28NCP
{'Current assets': 'nan', 'Cash at bank and in hand': '2', 'Net assets': '2', 'Issued share capital': 'nan', '2 Ordinary Shares 1 of Â£ each': '2', 'Total Shareholder funds': '2'}
^^^^^^^^^3^^^^^^^^^
X8Y28NOG
{'Called up share capital not paid': 'nan', 'Fixed Assets': 'nan', 'Current Assets': 'nan', 'Prepayments and accrued income': '

X8Y4UBL0
{'Called up share capital not paid': '3', 'Net assets': '3', 'Tssued share capital': 'nan', '3 Ordinary Shares 1 of Â£ each': '3', 'Total Shareholder funds': '3'}
^^^^^^^^^3^^^^^^^^^
X8Y4UBMG
^^^^^^^^^^0^^^^^^^^
X8Y4UC77
^^^^^^^^^^0^^^^^^^^
X8Y4UCJL
{'Called up share capital not paid': '2', 'Net assets': '2', 'Tssued share capital': 'nan', '2 Ordinary Shares 1 of Â£ each': '2', 'Total Shareholder funds': '2'}
^^^^^^^^^3^^^^^^^^^
X8Y4UCQQ
{'Fixed Assets': '157853', 'Current Assets': '5354', 'Creditors: amounts falling due within one year': '-614', 'Net current assets (liabilities)': '4740', 'Total assets less current liabilities': '162593', 'Creditors: amounts falling due after more than one': 'nan', '(10.469)': 'nan', 'year': 'nan', 'Total net assets (liabilities)': '152124', 'Capital and reserves': '152124'}
^^^^^^^^^3^^^^^^^^^
X8Y4UDKA
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each':

{'Current assets': '9106', 'Creditors: amounts falling due within': 'nan', 'one year': '-9132', 'Net current liabilities': '-26', 'Total assets less current liabilities': '-26', 'Net liabilities': '-26', 'Capital and reserves': '-26'}
^^^^^^^^^3^^^^^^^^^
X8Y4V0F5
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8Y4V15T
{'Fixed Assets': '3300', 'Current Assets': '2300', 'Creditors: amounts falling due within one year': '-3.361', 'Net current assets (liabilities)': '-1061', 'Total assets less current liabilities': '2239', 'Total net assets (liabilities)': '2239', 'Capital and reserves': '2239'}
^^^^^^^^^3^^^^^^^^^
X8Y4V20P
{'Current Assets': '23186', 'Creditors: amounts falling due within one year': '-7558', 'Net current assets (liabilities)': '15628', 'Total assets less current liabilities': '15628', 'Total net assets (liabilities)': '15628', 'Capital and

{'Current Assets': '10582', 'Creditors: amounts falling due within one year': '-10579', 'Net current assets (liabilities)': '3', 'Total assets less current liabilities': '3', 'Total net assets (liabilities)': '3', 'Reserves': '3'}
^^^^^^^^^3^^^^^^^^^
X8Y4VTZ6
{'Current assets': 'nan', 'Cash at bank and in hand': '4', 'Net assets': '4', 'Issued share capital': 'nan', '4 ordinary Shares 1 of Â£ each': '4', 'Total Shareholder funds': '4'}
^^^^^^^^^3^^^^^^^^^
X8Y4W2I3
{'Net assets': '0', 'Reserves': '0'}
^^^^^^^^^3^^^^^^^^^
X8Y4W2YH
^^^^^^^^^^0^^^^^^^^
X8Y4W2YW
{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', '100 Ordinary Shares of Â£ | each': '100', 'Total Shareholder funds': '100'}
^^^^^^^^^3^^^^^^^^^
X8Y4W3ZF
^^^^^^^^^^0^^^^^^^^
X8Y4W423
{'Fixed Assets': '290', 'Current Assets': '2090', 'Creditors: amounts falling due within one year': '-568', 'Net current assets (liabilities)': '1522', 'Total assets less current liabilities': '1812', 'Acc

X8Y4T6GG
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
^^^^^^^^^3^^^^^^^^^
X8Y4T6RU
{'Fixed Assets': '325705', 'Current Assets': '209051', 'Creditors: amounts falling due within one year': '-281290', 'Net current assets (liabilities)': '-72239', 'Total assets less current liabilities': '253466', 'Total net assets (liabilities)': '253466', 'Capital and reserves': '253466'}
^^^^^^^^^3^^^^^^^^^
X8Y4T6W9
{'Fixed Assets': '2844', 'Current Assets': '11230', 'Creditors: amounts falling due within one year': '-13192', 'Net current assets (liabilities)': '-1962', 'Total assets less current liabilities': '882', 'Total net assets (liabilities)': '882', 'Capital and reserves': '882'}
^^^^^^^^^3^^^^^^^^^
X8Y4T8NS
{'Net assets': '0', 'Reserves': '0'}
^^^^^^^^^3^^^^^^^^^
X8Y4T8P4
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share

## Observations

1) My code is not being able to extract information from all the txt files.(i.e. datas from few files were not extracted)                  
2) Many of my extracted datas are not being able to convert into scores. 

## Conclusion 

This is my first Machine Learning Challenge in which i have participated, though my score is not good but i went too close to get my result correct which is good for me. I had learnt a lot of new stuffs like regular expression, jupyter notebook, json and lot more.    

## About Myself

I am B.tech 3rd year student of NIT Durgapur. I am a competitive coder working on c/c++/python. In recent time i had worked on few ML projects in summer internship at IIT Kharagpur, where i worked on unsupervised learning(i.e. ICA,PCA,SVD..and lot more). After that i found intrest in this field and trying to explore... I would be honoured to get any project under your guidance in summer 2020, to gain experience and knowledge through the project. Hoping for your kind consideration.
